In [121]:
import requests
import pandas as pd
import numpy as np
import re

In [122]:
# цель - анализ финансовых показателей компаний в Свердловской области среди разработчиков информационных технологий
#

# Создание списка компаний

In [123]:
url1 = 'https://www.testfirm.ru/'

## 1 вариант - Список из первых 50 фирм - с первой страницы фильтра

In [124]:
url_list = 'https://www.testfirm.ru/rating' # рейтинг по всем предприятиям / #'https://www.testfirm.ru/rating/63/sverdlovskaya-oblast/'
k_1 = 63   #код деятельности
k_2 = "sverdlovskaya-oblast"  #область
url_list_copm = url_list + '/' + str(k_1) + '/' + k_2 + '/'
text = requests.get(url_list_copm) # 
try:
  text.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [125]:
# поиск названия компании
text_write = text.text

def find_komp(text_write, N):
    """ф. для поиска ИНН и названия по каждой компании - для формирования url-ссылки на информацию по этой компании
    """
    firma_list = text_write.split('org-link" href="/result/')
    return firma_list[N].split('"')[0]

List_all_komp = [] # список для url для каждой компании
for i in range(1, 51): # страница содержит список из 50 компаний на странице
    b = find_komp(text_write, i) 
    List_all_komp.append(b) # 
    print(i,'\t', List_all_komp[-1])
def find_komp_re(text_write, N):
  """  ф. для поиска ИНН и названия по каждой компании - с помощью регулярных выражений
  """
  pass    

1 	 6671042458_ooo-bit
2 	 6671332911_ao-regionalnyy-informatsionnyy-tsentr
3 	 6670320529_ooo-miralodzhik-it
4 	 6673240328_ooo-sertum-pro
5 	 6672258403_ooo-tekhkonsur-ooo-tekhkonsur
6 	 6673204560_ooo-ekspertbizneskonsalting
7 	 6670020959_ooo-kae-ekspert
8 	 6674301774_ooo-kompaniya-informatsionnykh-tekhnologiy
9 	 6660075589_ooo-siams
10 	 6678074441_ooo-ekstensiv-servis
11 	 6670419574_ooo-smartpeydzh
12 	 6659208481_ooo-boksberri-region
13 	 6671218373_ooo-konsultantplyus-ekaterinburg
14 	 6670237020_ooo-ntssoft
15 	 6671291239_ao-informatsionnaya-set
16 	 6671390568_ooo-plm-inzhiniring
17 	 6671341000_ooo-praymer
18 	 6658372383_ooo-konsultantplyusservis
19 	 6671142678_ooo-internet-pro
20 	 6625030240_pervouralskoe-mup-edinyy-raschetnyy-tsentr
21 	 6679077822_ooo-keysistems-ural
22 	 6670418517_ooo-profbizneskonsalting
23 	 6673174629_ooo-konsultant-sverdlovskaya-oblast
24 	 6674238593_ooo-konsultantplyus-sverdlovskaya-oblast
25 	 6672185480_ooo-ayteks
26 	 6658280140_ooo-stra

## 2 вариант - Список из всех фирм фильтра

### - загрузим страницу с рейтингом компаний

In [126]:
# Рейтинг организаций по выручке - Вид деятельности: 63 "Деятельность в области информационных технологий"
url_list_all = url1 + 'rating/63/' # адрес страницы с рейтингом
text_list_all = requests.get(url_list_all) # 
try:
  text_list_all.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [127]:
def find_name(url):
    """ ф. находит инфу для url страницы с информацией по компании - состоит из ИНН и названия Компании
    -  inn_firma 
    """
    text = requests.get(url)
    text_find = text.text 
    text_find = text_find.split('class="org-link"')[1].split('result/')[1].split('">')[0]
    return text_find

def find_name_next(url, koef = 1):
    """ ф. находит инфу для url страницы с информацией по следующей компании
    """
    text = requests.get(url)
    text_find = text.text 
    text_find = text_find.split('class="org-link"')[koef].split('result/')[1].split('">')[0]

    return text_find  

In [128]:
#url_one = 'https://www.testfirm.ru/result/' + name #   https://www.testfirm.ru/result/7708119944_ooo-sibirskaya-internet-kompaniya

def find_info_comp(url_one):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку
    """
    stroka = [] # собираем информацию сюда

    text_one = requests.get(url_one)
    text_write = text_one.text
    a1 = text_write.split('Организация: ')[1]
 #   stroka.append(a1.replace(' &quot;', '').split('<')[0].split(' ')[0]) # правовая форма
    if text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0]: # название
        stroka.append(text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0])
    else:
        stroka.append(0)
#    print(stroka[-1])
    stroka.append(a1.split('ИНН: ')[1][:10]) # ИНН
#    print(stroka[-1])
    stroka.append(a1.split('<b>')[2].split('<')[0].rstrip()) # активы
#    print(stroka[-1])
    stroka.append(a1.split('Активы на ')[1].split('/b>')[1][:5].rstrip()) # млн или тыс
#    print(stroka[-1])
    stroka.append(a1.split('Выручка')[1].split('b>')[1].split('<')[0].rstrip()) # выручка
#    print(stroka[-1])
    stroka.append(a1.split('Выручка')[1].split('b>')[2].split('руб')[0].rstrip()) # млн ии тыс
#    print(stroka[-1])
    if 'численности ' in text_write:
        stroka.append(a1.split('численности ')[1].split('b>')[1].split('<')[0]) # численность человек
#        print(stroka[-1])
    else:
        stroka.append(0) 
    if 'Дополнительные данные ФНС' in text_write:
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[1].split('<')[0].rstrip()) # доходы 
#        print(stroka[-1])
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[2].split('<')[0].rstrip()) # расходы
#        print(stroka[-1])
    else:
        stroka.append(0)
        stroka.append(0)
    if "Уплачено налогов" in text_write:
        stroka.append(text_write.split("Уплачено налогов")[1].split('b>')[1].split('<')[0].rstrip()) # налоги уплаченные
    else:
        stroka.append(0)
    return stroka

find_info_comp(url_one)

['"Сибирская Интернет Компания" ',
 '7708119944',
 '47 403',
 'млн.',
 '52 074',
 'млн.',
 '0',
 '0',
 '0',
 0]

In [ ]:
# проверка
Table_find_values = [] 

name = find_name(url_list_all)      # ф. найдет определитель компании - inn_firma
url_one = url1 + 'result/' + name   # url страницы с информацией о компании
text_one = requests.get(url_one)    # создат объект Response
a = find_info_comp(url_one)             # ф. покажет инф. о компании
print(a)
Table_find_values.append(a)
inn = name.split('_')[0]            # выделить инн
url_1 = url1 + 'rating/63_11_1/' + inn + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм
print('\t \t 1')

name_next_1 = requests.get(url_1).text.split(name)[1].split('href="/result/')[1].split('">')[0] # найти name следующей компании
inn_next = name_next_1.split('_')[0]
url_next_company = url1 + 'result/' + name_next_1
a = find_info_comp(url_next_company) #  ф. покажет инф. о компании
print(a)
Table_find_values.append(a)
url_1 = url1 + 'rating/63_11_1/' + inn_next + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм
print('\t \t 2')

for i in range(10):
    name_next_1 = requests.get(url_1).text.split(name_next_1)[1].split('href="/result/')[1].split('">')[0] # найти name следующей компании
    inn_next = name_next_1.split('_')[0]
    url_next_company = url1 + 'result/' + name_next_1
    a = find_info_comp(url_next_company) #  ф. покажет инф. о компании
    print(a)
    Table_find_values.append(a)
    url_1 = url1 + 'rating/63_11_1/' + inn_next + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм 
    print('\t \t', i + 3)

### - найдем информацию для поиска компании

### - определим функцию для поиска информации по каждой компании 

# Сбор данных по каждой компании

In [130]:
def find_info_comp_re(List_all_komp, n_comp):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку с помощью регулярных выражений
    """
    pass


# Анализ данных

In [131]:
data = pd.DataFrame(Table_find_values)
data.columns = ['Наименование', 'ИНН', 'Активы', 'ед.изм.', 'Выручка', 'ед.изм.', 'Численность персонала', 'Доходы', 'Расходы', 'Налоги'] 
data.head()

,Наименование,ИНН,Активы,ед.изм.,Выручка,ед.изм.,Численность персонала,Доходы,Расходы,Налоги
0,"""Сибирская Интернет Компания""",7708119944,47 403,млн.,52 074,млн.,0,0,0,0
1,"""МЭЙЛ РУ""",7743001840,198 742,млн.,45 582,млн.,0,0,0,0
2,"""Центр программ лояльности""",7702770003,8 723,млн.,32 778,млн.,166,33 174,30 617,1 680
3,"""Газпром информ""",7727696104,8 040,млн.,12 093,млн.,2135,12 138,11 819,1 870
4,"""ЛУКОЙЛ-Технологии""",9709038314,11 484,млн.,8 141,млн.,0,0,0,0


In [132]:
data.shape

(202, 10)